# Arboreto Tutorial

This notebook is a tutorial on how to use Arboreto to infer gene regulatory networks (GRN) from gene expression data.

## Contents

1. [Arboreto architecture](#1-Arboreto-architecture)
2. [Arboreto GRN inference algorithms](#2-Arboreto-GRN-inference-algorithms)
3. [Setup in Jupyter Notebook or JupyterLab](#3-Setup-in-Jupyter-Notebook-or-JupyterLab)
4. [Arboreto API](#4-Arboreto-API)

---
# 1 Arboreto architecture

Arboreto is a computational framework that hosts GRN inference algorithms that follow the __*"Multiple Ensemble"*__ inference strategy, of which [GENIE3](http://www.montefiore.ulg.ac.be/~huynh-thu/GENIE3.html) is probably the most well-known example. 

> *This method approaches the network inference problem by decomposing
it into a separate regression problem for each possible target
gene. Next, using a tree-based ensemble method, an importance
measure for each predictor is calculated and a high feature
importance is used as an indication that a link is present between
the predictor and the target gene in the GRN.*

> Ref: [NIMEFI: Gene Regulatory Network Inference using Multiple Ensemble Feature Importance Algorithms](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0092709). Ruyssinck et al. (2014) PLoS ONE

It is often mentioned that the inference strategy of computing a regression per target is highly parallelizable, and can therefore by run in parallel on multiple compute nodes in a cluster. Arboreto answers this call by representing this computation in a [Dask](https://dask.pydata.org/en/latest/install.html) computation graph, enabling the computational workload to scale out over multiple CPU cores (scaling *up*) as well as over multiple compute nodes (scaling *out*).

Arboreto uses only a small subset of the Dask API: Dask [delayed](https://dask.pydata.org/en/latest/delayed.html) functions and Dask [DataFrames](https://dask.pydata.org/en/latest/dataframe.html). Arboreto uses these building blocks to create a [custom graph](https://dask.pydata.org/en/latest/custom-graphs.html). A [Dask](https://dask.pydata.org/en/latest/install.html) graph can be regarded as a *recipe* that is executed by a Dask [scheduler](https://dask.pydata.org/en/latest/scheduler-overview.html), the Dask component that orchestrates the computational tasks.

As a user of Arboreto, it is not necessary to master these concepts, they remain hidden behind the [Arboreto API](#4-Arboreto-API).

---
# 2 Arboreto GRN inference algorithms

## 2.1 GRNBoost2
Arboreto implements GRNBoost2, an improved version of the [GRNBoost](https://github.com/aertslab/GRNBoost/) algorithm that was originally built on [Xgboost](https://xgboost.readthedocs.io/en/latest/), using [Apache Spark](http://spark.apache.org/) as the computation engine. 

GRNBoost2 uses the scikit-learn [GradientBoostingRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html). GRNBoost2 uses [early-stopping](https://en.wikipedia.org/wiki/Early_stopping) regularization and a modified feature importances normalization that compensates for the issue of comparing regression ensembles of different sizes when composing a GRN from the feature importances of different regressions.

## 2.2 GENIE3
Arboreto implements the GENIE3 specification by providing the GENIE3 recommended parameters for the scikit-learn [RandomForestRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html).


---
# 3 Setup in Jupyter Notebook or JupyterLab

It is recommended to use a recent (TODO: versions) [Anaconda](https://docs.anaconda.com/) or [Miniconda](https://conda.io/miniconda.html) distribution.

## 3.1 Arboreto dependencies
* [numpy](https://anaconda.org/anaconda/numpy)
* [scikit-learn](http://scikit-learn.org/stable/install.html)
* [pandas](https://pandas.pydata.org/pandas-docs/stable/install.html)
* [dask](https://dask.pydata.org/en/latest/install.html) and optionally [dask-distributed](http://distributed.readthedocs.io/en/latest/install.html)

## 3.2 Setup from source

Arboreto is very lightweight python package that can be used from a [Jupyter Notebook](http://jupyter.org/) or [JupyterLab](https://github.com/jupyterlab/jupyterlab) environment by importing the `core.py` file.

```
$ git clone https://github.com/tmoerman/arboreto
$ cd arboreto/notebooks
$ jupyter lab
```

First, add the Arboreto source file to the `sys.path`:

In [6]:
import sys

sys.path.append('../')  # path to the arboreto root folder

from arboreto.core import *
from arboreto.utils import *

The Arboreto functions are now available in the notebook.

## 3.3 Setup with Pip

TODO

---
# 4 Arboreto API

Arboreto aims at providing a minimal API, using only common python idioms and data structures.

## 4.1 Creating a Dask computation graph

The heart of Arboreto is the `create_graph` function that creates the Dask computation graph in function of the input data, the regressor parameters and a few optional parameters that are unnecessary in typical use.

Example:

```python
graph = create_graph(zeisel_ex_matrix,
                     zeisel_gene_names,
                     zeisel_tf_names,
                     "GBM",
                     SGBM_KWARGS)
```

You can use the question mark `?` in Jupyter to inspect a function's documentation.

In [5]:
create_graph?

Signature: create_graph(expression_matrix, gene_names, tf_names, regressor_type, regressor_kwargs, target_genes='all', limit=None, include_meta=False, early_stop_window_length=25, seed=666)
Docstring:
Main API function. Create a Dask computation graph.

:param expression_matrix: numpy matrix. Rows are observations and columns are genes.
:param gene_names: list of gene names. Each entry corresponds to the expression_matrix column with same index.
:param tf_names: list of transcription factor names. Should have a non-empty intersection with gene_names.
:param regressor_type: regressor type. Case insensitive.
:param regressor_kwargs: dict of key-value pairs that configures the regressor.
:param target_genes: either int, 'all' or a collection that is a subset of gene_names.
:param limit: int or None. Default 100k. The number of top regulatory links to return.
:param include_meta: Also return the meta DataFrame. Default False.
:param early_stop_window_length: window length of the early stop

### 4.1.1 Input parameters

Arboreto expects the input data in a specific shape

#### `expression_matrix`

* A [numpy matrix](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.matrix.html) where the __rows are observations__ and the __columns are genes__.

> **CAUTION **

> This is contrary to common practice in biology where the rows are the genes and the columns are the observations (e.g. cells). 
> We chose the orientation that is expected by the scikit-learn regressors to keep the library as minimal as possible. It is the responsibility of the user to provide the expression matrix in the correct orientation.

#### `gene_names`

* A list of gene names, in the same order as the columns of the `expression_matrix`. 
* Consider it the header of the numpy matrix.

#### `tf_names`

* A list of transcription factor (predictor) names. No particular ordering is required. 
* Arboreto expects that there is a non-empty intersection between `gene_names` and `tf_names`.

#### `regressor_type`

* One of `["RF", "GBM", "ET"]`: specifies which scikit-learn regressor to use:


| type  | regressor | algorithm |
| ---   | ---       | ---         |
| `RF`  | `RandomForestRegressor` | GENIE3 default with Random Forest |
| `ET`  | `ExtraTreesRegressor`   | GENIE3 alternative with Extra-Trees |
| `GBM` | `GradientBoostingRegressor` | GRNBoost2  |

#### `regressor_kwargs`

* A python dictionary of keyword-argument pairs (hence kwargs) to configure the scikit-learn regressor.

### 4.2 Computing the GRN from a Dask graph

As mentioned earlier, a Dask graph does not perform the computations, it is merely the computational *recipe* intepreted and executed by a Dask [scheduler](https://dask.pydata.org/en/latest/scheduler-choice.html).

Arboreto is typically used with the Dask distributed scheduler, which can be used to run the computations with multiple processes on a single machine or using multiple machines connected to the scheduler.

#### Computing on a single machine (easy)